In [ ]:
%load_ext autoreload
%autoreload 2

#  Sliceguard – Find critical data segments in your data (fast)
## Structured Data Walkthrough

Sliceguard is a python library for quickly finding **critical data slices** like outliers, errors, or biases. It works on **structured** and **unstructured** data.

This notebook showcases especially the **structured** data case, so if you have unstructured data like images or audio instead have a look at **[guide for unstructured data](./quickstart_unstructured_data.ipynb)** instead

It is interesting for you if you want to do the following:
1. Find **performance issues** of your machine learning model.
2. Find **anomalies and inconsistencies** in your data.
3. Quickly **explore** your data using an interactive report to generate **insights**.

To run this notebook install and import sliceguard:

In [ ]:
!pip install sliceguard

In [ ]:
from sliceguard import SliceGuard
from sliceguard.data import from_huggingface
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

Now download the demo dataset from the huggingface hub:

In [ ]:
df = from_huggingface("mstz/wine")

In [ ]:
# Define the feature names
feature_names = ["fixed_acidity", "volatile_acidity", "citric_acid", "residual_sugar",
                "chlorides", "free_sulfur_dioxide", "total_sulfur_dioxide", "density", "pH",
                "sulphates", "alcohol"]

In [ ]:
# Show dataframe
df

## Check for data slices that are particulary different (Outliers/Errors in the data)
Here sliceguard will train an **outlier detection** model to highlight data points that are especially different from the rest. Note that we here run the outlier detection only on the class "red". You will easily see there is a bunch of large outliers e.g. with extremely high residual sugar values.

You can use the **report feature** that uses [Renumics Spotlight](https://github.com/Renumics/spotlight) for visualization to dig into the reasons why a cluster is considered an outlier. E.g. use multiple **histograms** to see what is characteristic for a datapoint.

In [ ]:
sg = SliceGuard()
issues = sg.find_issues(df[df["is_red"] == "red"], features=feature_names)

In [ ]:
_ = sg.report()

## Check for data slices where models are prone to fail (hard samples, inconsistencies)
Here sliceguard will **train a classification model** and check for data slices where the accuracy score is particulary bad. You will realize that this will show you a bunch of wines that are easily confused using there chemical values, e.g. red wines with particulary high residual sugar levels or especially low chloride values.

Note that the dataset is relatively easy, so it won't find that many issues as in more complex datasets.

In [ ]:
# Train the model and predict on the same data (of course in practice you will want to split your data!!!)
# This is only for showing the principle
sg = SliceGuard()
issues = sg.find_issues(df,
                        features=feature_names,
                        y="is_red",
                        metric=accuracy_score,
                        automl_task="classification"
                       ) # also try out drop_reference="parent" for more class-specific results

In [ ]:
_ = sg.report()

## Check for weaknesses of your own model (...and hard samples + inconsistencies)
This shows how to pass your **own model predictions** into sliceguard to find slices that are performing badly according to a supplied metric function. This allows you to uncover **inconsistencies** and samples that are **hard to learn** in no time!

In [ ]:
# Train the model and predict on the same data (of course in practice you will want to split your data!!!)
# This is only for showing the principle
X = StandardScaler().fit_transform(df[feature_names].values)
y = df["is_red"]

clf = SVC()
clf.fit(X, y)
df["predictions"] = clf.predict(X)

In [ ]:
# Pass the predictions to sliceguard and uncover hard samples and inconsistencies.
sg = SliceGuard()
issues = sg.find_issues(df,
                        features=feature_names,
                        y="is_red",
                        y_pred="predictions",
                        metric=accuracy_score)

In [ ]:
_ = sg.report()